# Basics

In [ ]:
# !pip3 install pythainlp
# !pip3 install https://github.com/PyThaiNLP/thai_sentiment_analysis/archive/master.zip
# !pip3 install kenlm==0.2.0
# !pip3 install pypdf==3.17.1
# !pip3 install pytesseract==0.3.10
# !pip3 install PyMuPDF==1.23.6
# !pip3 install transformers==4.35.2

In [ ]:
from pythainlp import word_tokenize, Tokenizer

text = "สมชายเห็นชอบกลบทบาทนี้"

print("newmm  :", word_tokenize(text))
print("longest:", word_tokenize(text, engine="longest"))

# Computational Linguistics

## Reverse Dictionary

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = {
    "Word": ["แล", "เบิ่ง", "ผ่อ"],
    "POS" : ["ก.", "ก.", "ก."],
    "Definition": ["ดู มอง", "ดู มอง เหลียวดู", "ดู ดูแล มอง"]
}

df = pd.DataFrame(data)

def calculate_cosine_similarity(definitions):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(definitions)
    cosine_sim = cosine_similarity(tfidf_matrix)
    return cosine_sim

def pos_similarity(pos_list):
    n = len(pos_list)
    total_sim = 0
    count = 0
    for i in range(n):
        for j in range(i+1, n):
            total_sim += 1 if pos_list[i] == pos_list[j] else 0
            count += 1
    return total_sim / count if count > 0 else 0

def definition_similarity(definitions):
    similarity_matrix = calculate_cosine_similarity(definitions)
    n = similarity_matrix.shape[0]
    total_sim = 0
    count = 0
    for i in range(n):
        for j in range(i+1, n):
            total_sim += similarity_matrix[i, j]
            count += 1
    return total_sim / count if count > 0 else 0

alpha = 0.5
beta = 0.5

pos = df["POS"].values
pos_sim = pos_similarity(pos)

definitions = df["Definition"]
def_sim = definition_similarity(definitions)
similarity = alpha * pos_sim + beta * def_sim

print(f"POS Similarity: {pos_sim:.2f}")
print(f"Definition Similarity: {def_sim:.2f}")
print(f"Overall Similarity: {similarity:.2f}")

## TF-IDF

In [ ]:
import math
from collections import Counter
from typing import List, Dict

class TFIDFCalculator:
    def __init__(self, documents: List[List[str]]):
        self.documents = documents
        self.doc_count = len(documents)
        self.term_freq = [Counter(doc) for doc in documents]
        self.doc_lengths = [len(doc) for doc in documents]

    def calculate_tf(self, term: str, doc_idx: int) -> float:
        if self.doc_lengths[doc_idx] == 0:
            return 0
        return self.term_freq[doc_idx][term] / self.doc_lengths[doc_idx]

    def calculate_idf(self, term: str) -> float:
        doc_with_term = sum(1 for doc in self.documents if term in doc)
        if doc_with_term == 0:
            return 0
        return math.log2(self.doc_count / doc_with_term)

    def calculate_tfidf(self, terms: List[str]) -> Dict[str, List[float]]:
        results = {}
        for term in terms:
            idf = self.calculate_idf(term)
            tfidf_scores = [
                round(self.calculate_tf(term, doc_idx) * idf, 4)
                for doc_idx in range(self.doc_count)
            ]
            results[term] = tfidf_scores
        return results

def main():
    documents = [
        ["นวัตกรรม", "พลังงาน", "สะอาด", "เพื่อ", "โลก",
        "ยั่งยืน", "พลังงาน", "แสงอาทิตย์", "และ", "ลม",
        "กำลัง", "เป็นที่นิยม", "ใน", "ประเทศไทย", "นักวิทยาศาสตร์",
        "คาดว่า", "จะ", "ช่วย", "ลด", "การปล่อย",
        "ก๊าซ", "เรือนกระจก", "ได้", "อย่างมาก"],
        ["เศรษฐกิจ", "ไทย", "ฟื้นตัว", "หลัง", "โควิด",
        "การท่องเที่ยว", "และ", "การส่งออก", "เป็น", "ปัจจัย",
        "สำคัญ", "ใน", "การ", "ขับเคลื่อน", "เศรษฐกิจ",
        "รัฐบาล", "เร่ง", "ออก", "มาตรการ", "กระตุ้น"],
        ["นวัตกรรม", "ปัญญาประดิษฐ์", "ใน", "วงการ", "แพทย์",
        "AI", "ช่วย", "วินิจฉัย", "โรค", "ได้",
        "แม่นยำ", "ขึ้น", "โรงพยาบาล", "ใน", "ประเทศไทย",
        "เริ่ม", "นำ", "มา", "ใช้"],
        ["การเปลี่ยนแปลง", "สภาพ", "ภูมิอากาศ", "กระทบ", "ภาค",
        "เกษตร", "เกษตรกร", "ไทย", "ปรับตัว", "รับมือ",
        "ภัยแล้ง", "และ", "น้ำท่วม", "นักวิทยาศาสตร์", "เร่ง",
         "คิดค้น", "พันธุ์พืช", "ทนทาน"],
        ["พลังงาน", "นิวเคลียร์", "ทางเลือก", "หรือ", "ทางตัน",
        "ประเทศไทย", "ยัง", "ลังเล", "ใน", "การพัฒนา",
        "โรงไฟฟ้า", "นิวเคลียร์", "ขณะที่", "หลาย", "ประเทศ",
        "เดินหน้า", "เต็มที"],
        ["การพัฒนา", "เมือง", "อัจฉริยะ", "ใน", "ประเทศไทย",
        "กรุงเทพฯ", "และ", "เมือง", "ใหม่", "เร่ง",
        "ปรับตัว", "สู่", "Smart City", "ใช้", "เทคโนโลยี",
        "IoT", "เพื่อ", "ยกระดับ", "คุณภาพ", "ชีวิต"],
        ["วิกฤต", "ขยะ", "พลาสติก", "ใน", "ทะเลไทย",
        "นักวิทยาศาสตร์", "เตือน", "ผลกระทบ", "ต่อ", "ระบบนิเวศ",
        "รัฐบาล", "ออก", "มาตรการ", "ลด", "การใช้",
        "พลาสติก"],
        ["5G", "เปลี่ยน", "โฉม", "อุตสาหกรรม", "ไทย",
        "ผู้ประกอบการ", "เร่ง", "ปรับตัว", "รับ", "เทคโนโลยี",
        "ใหม่",  "คาด", "ช่วย", "เพิ่ม", "ประสิทธิภาพ",
        "การผลิต"],
        ["การท่องเที่ยว", "เชิงนิเวศ", "บูม", "ใน", "ไทย",
        "นักท่องเที่ยว", "ต่างชาติ", "สนใจ", "ธรรมชาติ", "และ",
        "วัฒนธรรม", "ท่องถิ่น", "ช่วย", "กระจาย", "รายได้",
        "สู่", "ชุมชน"],
        ["พลังงาน", "สะอาด", "กับ", "การพัฒนา", "ที่",
        "ยั่งยืน", "ประเทศไทย", "ตั้งเป้า", "เพิ่ม", "สัดส่วน",
        "พลังงาน", "หมุนเวียน", "นักลงทุน", "สนใจ","ลงทุน",
        "ใน", "โครงการ", "พลังงาน", "แสงอาทิตย์", "และ",
        "ลม"]
    ]
    target_words = ["พลังงาน", "นวัตกรรม", "เศรษฐกิจ", "ประเทศไทย", "เทคโนโลยี"]
    calculator = TFIDFCalculator(documents)
    results = calculator.calculate_tfidf(target_words)

    for term, scores in results.items():
        print(f"\nคำว่า '{term}':")
        for doc_idx, score in enumerate(scores, 1):
            if score > 0:
                print(f"D{doc_idx}: {score:.4f}")

if __name__ == "__main__":
    main()

## Cosine Similarity

In [ ]:
import numpy as np
from numpy.linalg import norm

A = np.array([
  2, 1, 2, 3, 2, 9
  ])

B = np.array([
  3, 4, 2, 4, 5, 5
  ])

print("A:", A)
print("B:", B)

cosine = np.dot(A,B)/(norm(A)*norm(B))
print(f"Cosine Similarity: {cosine:.4f}")

## Word Embedding

In [ ]:
# cosine similarity and distance

import pandas as pd
import numpy as np

rows = ["น้ำ", "ข้าว", "ผลไม้", "จาน", "แก้ว", "เนื้อ", "ปลา", "ผัก"]
columns = ["กิน", "ดื่ม", "ซื้อ", "ล้าง", "เก็บ", "ปรุง", "หั่น", "แช่", "ขวด",
           "ถาด", "ชาม", "ถ้วย", "ชิ้น", "ผล", "ใน", "บน", "กับ", "และ"]

data = [[25, 95, 42, 38, 12, 0, 0, 85, 90, 0, 0, 65, 0, 0, 75, 0, 85, 45],
        [82, 0, 35, 0, 45, 58, 0, 0, 0, 85, 90, 75, 0, 0, 65, 0, 78, 55],
        [68, 52, 73, 45, 38, 0, 75, 65, 0, 25, 0, 0, 85, 95, 45, 0, 65, 85],
        [0, 0, 28, 92, 85, 0, 0, 0, 0, 0, 72, 0, 0, 0, 0, 95, 45, 55],
        [0, 88, 32, 75, 62, 0, 0, 0, 87, 0, 0, 0, 0, 0, 0, 85, 35, 45],
        [81, 0, 60, 68, 56, 72, 85, 55, 0, 75, 65, 0, 95, 0, 45, 75, 85, 65],
        [85, 0, 65, 72, 48, 78, 82, 62, 0, 78, 68, 0, 92, 0, 52, 72, 88, 58],
        [75, 0, 70, 85, 52, 65, 88, 45, 0, 65, 55, 0, 0, 0, 35, 65, 92, 75]]

df = pd.DataFrame(data, index=rows, columns=columns)

def cosine_calculation(word_pair, mode = "similarity"):
    vec1 = df.loc[word_pair[0]].values
    vec2 = df.loc[word_pair[1]].values
    dot_product = np.dot(vec1, vec2)
    magnitude1 = np.linalg.norm(vec1)
    magnitude2 = np.linalg.norm(vec2)
    cosine_similarity = dot_product / (magnitude1 * magnitude2)
    result = cosine_similarity if mode == "similarity" else 1 - cosine_similarity

    return f"cosine {mode} ระหว่าง '{word_pair[0]}' และ '{word_pair[1]}' = {result:.4f}"

# print(cosine_calculation(("จาน", "ปลา")))
# print(cosine_calculation(("เนื้อ", "น้ำ")))
# print(cosine_calculation(("ผัก", "ข้าว")))
# print(cosine_calculation(("ผลไม้", "แก้ว"), mode = "distance"))
# print(cosine_calculation(("ข้าว", "แก้ว"), mode = "distance"))
# print(cosine_calculation(("ปลา", "แก้ว"), mode = "distance"))

In [ ]:
# การวิเคราะห์ความสัมพันธ์ระหว่างคำนามกับลักษณนาม

classifiers = ["ขวด", "ถาด", "ชาม", "ถ้วย", "ชิ้น", "ผล"]
nouns = df.index

classifier_df = df[classifiers]

for classifier in classifiers:
    relation = classifier_df[classifier].max()
    nouns = classifier_df[classifier_df[classifier] == relation].index
    print(f"ลักษณนาม '{classifier}' มีความสัมพันธ์กับ \"{','.join(nouns)}\" สูงสุด - {relation}")

In [ ]:
# เปรียบเทียบการใช้ "ใน" และ "บน"

prepositions = ["ใน", "บน"]
for prep in prepositions:
    sorted_values = df[prep].sort_values(ascending=False)
    print(f"\nการใช้ '{prep}':")
    for noun, value in sorted_values.items():
        if value > 0:
            print(f"{noun}: {value}")

In [ ]:
# การจัดกลุ่มคำนามที่มีความสัมพันธ์ใกล้เคียง

from itertools import combinations

# คำนวณ cosine similarity
def mini_cosine(word1, word2):
    vec1 = df.loc[word1].values
    vec2 = df.loc[word2].values
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# หาความสัมพันธ์ระหว่างทุกคู่คำ
word_pairs = list(combinations(df.index, 2))
similarities = [(pair[0], pair[1], mini_cosine(pair[0], pair[1])) 
               for pair in word_pairs]

# เรียงลำดับและแสดงผล
similarities.sort(key=lambda x: x[2], reverse=True)
print("คู่คำที่มีความสัมพันธ์ใกล้เคียงที่สุด:")
for pair in similarities[:5]:
    print(f"{pair[0]} - {pair[1]}: {pair[2]:.4f}")

In [ ]:
# วิเคราะห์การใช้กริยา "กิน ดื่ม ปรุง หั่น"

verbs = ["กิน", "ดื่ม", "ปรุง", "หั่น"]
verb_df = df[verbs]
distances = []

for noun in df.index:
    current_vec = verb_df.loc[noun].values

    other_nouns = [n for n in df.index if n != noun]
    other_vectors = verb_df.loc[other_nouns].values
    mean_vec = np.mean(other_vectors, axis=0)
    
    distance = 1 - np.dot(current_vec, mean_vec) / (np.linalg.norm(current_vec) * np.linalg.norm(mean_vec))
    distances.append((noun, distance))

distances.sort(key=lambda x: x[1], reverse=True)

print("\nความแตกต่างของรูปแบบการใช้กริยา (เรียงจากมากไปน้อย):")
for noun, dist in distances:
    print(f"{noun}: {dist:.4f}")
    pattern = verb_df.loc[noun]
    # print(f"Pattern: กิน = {pattern['กิน']}, ดื่ม = {pattern['ดื่ม']}, ปรุง = {pattern['ปรุง']}, หั่น = {pattern['หั่น']}")

In [ ]:
# ประโยค "คุณแม่_น้ำใส่แก้ว"

target_verbs = df.columns[:8]  # ['กิน', 'ดื่ม', 'ซื้อ', 'ล้าง', 'เก็บ', 'ปรุง', 'หั่น', 'แช่']
scores = []

for verb in target_verbs:
    # คำนวณ score โดยคูณค่าความสัมพันธ์ของทั้งสองคำ
    score = df.loc["น้ำ", verb] * df.loc["แก้ว", verb]
    scores.append((verb, score))

# เรียงลำดับจากมากไปน้อย
scores.sort(key=lambda x: x[1], reverse=True)
print("คะแนนความเป็นไปได้ของแต่ละกริยา:")
for verb, score in scores:
    if score > 0:  # แสดงเฉพาะกริยาที่มีความเป็นไปได้ (score > 0)
        print(f"{verb}: {score}")

In [ ]:
# ความเป็นไปได้ของลักษณนามกับ "เนื้อ"

classifiers = ["ชิ้น", "ถาด", "ชาม"] # เลือกเฉพาะลักษณนามที่เกี่ยวข้อง

# แสดงค่าความสัมพันธ์ก่อนคำนวณความน่าจะเป็น
print("ค่าความสัมพันธ์ระหว่างคำว่า 'เนื้อ' กับลักษณนามต่างๆ:")
for clf in classifiers:
    print(f"{clf}: {df.loc['เนื้อ', clf]}")

# คำนวณผลรวมของค่าความสัมพันธ์ทั้งหมด
total = sum(df.loc["เนื้อ", classifiers])
print(f"ผลรวมค่าความสัมพันธ์: {total}")

classifier_probs = []
for clf in classifiers:
    # คำนวณความน่าจะเป็นโดยหารด้วยผลรวม
    prob = df.loc["เนื้อ", clf] / total
    classifier_probs.append((clf, prob))

# เรียงลำดับความน่าจะเป็นจากมากไปน้อย
classifier_probs.sort(key=lambda x: x[1], reverse=True)
print("\nความน่าจะเป็นของแต่ละลักษณนาม:")
for clf, prob in classifier_probs:
    print(f"{clf}: {prob:.4f} ({df.loc['เนื้อ', clf]}/{total})")

## Sentiment Analysis

### Project: Language in Digital Media
**LG468 Language in Digital Media**

In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pythainlp.corpus.common import thai_stopwords
from wordcloud import WordCloud, STOPWORDS

In [2]:
# API Configuration

API_KEY = 'kHIllIH4ODKsOvvi7QJINN5FIzf6sFgR'
API_FOR_THAI = "https://api.aiforthai.in.th"
SSSENSE_ENDPOINT = f"{API_FOR_THAI}/ssense"
TEXT_CLEANSING_ENDPOINT = f"{API_FOR_THAI}/textcleansing"

HEADERS = {"apikey": API_KEY}

In [ ]:
# Data Loading and Preprocessing

def load_data(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read().splitlines()
            return data
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []

def cleanse_data(data):
    cleaned_data = []
    for text in data:
        response = requests.post(TEXT_CLEANSING_ENDPOINT, data={'text': text}, headers=HEADERS)
        cleaned_data.append(response.json()['cleansing_text'])
    return cleaned_data

data = load_data(r'datasets\test.csv')

cleaned_data = cleanse_data(data)

['อ้าว อยากมีเรื่องหรอวะ ไอ้หัวเกรียน!', 'ขอโทษที่รบกวนคะ', 'รักเธอนะ คนดี', 'เหี้ย สัตว์มึงอย่าเสือกดิ', 'ไอ้คนทรยศ แกอย่าหวังว่าจะตายดี', 'ดีใจด้วยนะ คุณหญิงกฤตยา', 'เธอเป็นความภูมิใจที่สุดของฉัน', 'ทปอ. ว่าแต่เขาอิเหนาเป็นเอง ขี้คุยชมัด', 'ติดมหิดล รอบพอร์ตแล้ว เย้ๆๆ', 'มือถือค้างอ่ะ เซ็งจัง', 'เอาหมูคุโรบุตะมาทำสเต๊กนี่ กินแล้วฟินขั้นเทพพพพ', 'เกรซไม่เคยเปิดใจให้เรา ไม่รู้เค้ามีอคติไรนักหนา', 'สอบเกือบผ่าน ตกไปแค่ 0.5 คะแนนเอง ถถถถ', 'ทักคับ 😃', 'ยินดีด้วยแนนได้ผู้แทนเลขละ มาสอนข้าน้อยด้วยด่วนๆ5555', 'น่าเสียดายอ่ะน้องแกพลาดประธานตึกไปแค่ 16 แต้ม 😭', 'น่ารักมากมายด์ ตายอย่างสงบศพสีชมพู❤️', 'หลายครั้ง ความรักก็ทำให้เราตาบอดนะ เอิร์น', 'เหม็น หยุดใช้น้ำหอมยี่ห้อนี้ได้มะ เรารู้สึกเหม็นฉุนๆ มากกว่าหอมอ่า._.', 'พระคุณที่สามงดงามแจ่มใส', 'ของเก่าจนเกือบจะใช้ไม่ได้ เหมือนอายุเจ้าของที่ใกล้ลงโลง', 'สบายดีจ้า~', '5555555555 ขำจังเลย ตลกดี ชอบ', 'เราไม่ชอบเฌอปราง แบบเกลียดขี้หน้านาง', 'แล้วจะให้ฉันทำยังไง แกทำตัวแย่ซะขนาดนี้ ไอ้ลูกทรพี', 'หิวข้าว รู้งี้กินข้าวเที่ยงดีกว่า เรื่องจริงนาจา', 'ส

In [ ]:
# Sentiment Analysis

def analyze_sentiment(data):
    text = []
    polarity = []
    confidence = []
    keywords = []
    poswords = []
    negwords = []

    for text_data in data:
        response = requests.post(SSSENSE_ENDPOINT, data={'text': text_data}, headers=HEADERS)
        if response.json()['sentiment']['score'] > '50':
            text.append(response.json()['preprocess']['input'])
            polarity.append(response.json()['sentiment']['polarity'])
            confidence.append(float(response.json()['sentiment']['score']))
            keywords.extend(response.json()['preprocess']['keyword'])
            if response.json()['preprocess']['pos']:
                poswords.extend(response.json()['preprocess']['pos'])
            if response.json()['preprocess']['neg']:
                negwords.extend(response.json()['preprocess']['neg'])

    return text, polarity, confidence, keywords, poswords, negwords

text, polarity, confidence, keywords, poswords, negwords = analyze_sentiment(cleaned_data)

In [ ]:
# Data Processing and Output

def process_data(text, polarity, confidence):
    confidence_lst = list(zip(polarity, confidence))
    predicted_lst = list(zip(text, polarity))
    return confidence_lst, predicted_lst

confidence_lst, predicted_lst = process_data(text, polarity, confidence)

print(confidence_lst)
print(predicted_lst)

In [ ]:
df = pd.DataFrame(confidence_lst, columns=['Sentiment', 'Confidence'])

sns.set_theme(style="whitegrid")

plt.figure(figsize=(10, 5))
sns.boxplot(x='Sentiment', y='Confidence', data=df)
plt.title("Confidence Scores by Sentiment")
plt.xlabel("Sentiment")
plt.ylabel("Confidence (%)")
plt.show()

In [ ]:
df = pd.DataFrame(confidence_lst, columns=['Sentiment', 'Confidence'])

bins = np.linspace(50, 100, 10)

df['Confidence_Range'] = pd.cut(df['Confidence'], bins=bins, include_lowest=True)

pivot_df = df.pivot_table(values='Confidence', index='Confidence_Range', 
                          columns='Sentiment', aggfunc='count', fill_value=0)

pivot_df = pivot_df.sort_index(ascending=False)

plt.figure(figsize=(10, 8))
sns.heatmap(pivot_df, annot=False, cmap='YlOrRd', cbar_kws={'label': 'Count'})
plt.title("Confidence Scores by Sentiment")
plt.xlabel("Sentiment")
plt.ylabel("Confidence Score Ranges")
plt.tight_layout()
plt.show()

In [ ]:
# Plotting Word Clouds

text_neg = " ".join(text for text, sentiment in predicted_lst if sentiment == 'negative')
text_pos = " ".join(text for text, sentiment in predicted_lst if sentiment == 'positive')

fp = 'THSarabunNew.ttf'
reg = r"[ก-๙a-zA-Z']+"
thai_stopwords = list(thai_stopwords())

wordcloud_neg = WordCloud(stopwords=thai_stopwords, background_color='white', max_words=2000,
                          height=2000, width=4000, font_path=fp, regexp=reg).generate(text_neg)

wordcloud_pos = WordCloud(stopwords=thai_stopwords, background_color='white', max_words=2000,
                          height=2000, width=4000, font_path=fp, regexp=reg).generate(text_pos)

fig, axs = plt.subplots(1, 2, figsize=(16, 8))

axs[0].imshow(wordcloud_neg, interpolation='bilinear')
axs[0].axis('off')
axs[0].set_title('Negative Sentiment')

axs[1].imshow(wordcloud_pos, interpolation='bilinear')
axs[1].axis('off')
axs[1].set_title('Positive Sentiment')

plt.show()